<a href="https://colab.research.google.com/github/lhg1992/Tensorflow_Trials/blob/main/MyResNetforCifar10_Tensorflow_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReadMe

The augment dataset can't train up directly.
<br>However, it can be used after training the original dataset, and boost val_acc from 81% to 91%!
Strange! (sn=10)


# Initialization

In [1]:
import tensorflow as tf
import numpy as np


from tensorflow.keras import datasets, layers, activations, models
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,  Layer
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import ZeroPadding2D, Activation, Add
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop, RandomFlip, RandomRotation
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

import pickle
import os

import time
# start_time = time.time()

print(os.getcwd())
# tf.random.set_seed(sn)
sn=10

/content


In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Data Preparation

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Another Normalization for augmentation ??bad
# train_images=(train_images-[0.4914, 0.4822, 0.4465])/np.array([0.2023, 0.1994, 0.2010])
# test_images=(test_images-[0.4914, 0.4822, 0.4465])/np.array([0.2023, 0.1994, 0.2010])

print('traindata shape:'+ str(train_images.shape))
print('trainlabel shape:'+ str(train_labels.shape))
print('testdata shape:'+ str(test_images.shape))


class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
print(type(train_images))


# Augmentation
aug = ImageDataGenerator(horizontal_flip=True, width_shift_range=4,
                             height_shift_range=4)
aug.fit(train_images)
train_dataset=aug.flow(train_images,train_labels,batch_size = 128)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(128)

traindata shape:(50000, 32, 32, 3)
trainlabel shape:(50000, 1)
testdata shape:(10000, 32, 32, 3)
<class 'numpy.ndarray'>


## With TPU Compatibility

In [ ]:
# train_images_pad = tf.keras.layers.ZeroPadding2D(padding=4)(train_images)
# train_dataset = tf.data.Dataset.from_tensor_slices((train_images_pad, train_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
# test_dataset = test_dataset.batch(128)

# # sess = tf.compat.v1.Session()
# def data_augment(image, label):        
# #     image=tf.pad(image, [[4, 4], [4, 4]], "CONSTANT", 0) 
# # #     image = tf.keras.layers.ZeroPadding2D(padding=4)(image) # require  4D    
#     image = tf.image.random_crop(image, train_images.shape[1:], seed=sn) #require pad first
    
#     image = tf.image.random_flip_left_right(image,seed=sn)
# #     image = tf.keras.preprocessing.image.random_rotation(image.eval(session=sess), 20)# fail, also can't fit TPU
# #     image = tf.image.random_flip_up_down(image)
# #     image = tf.image.rot90(image, k=1) # rotate 90º
#     # image = tf.image.random_brightness(image, max_delta=.2) # shouldn't use,
#     return image, label


# train_dataset = train_dataset.map(data_augment)
# # #train_dataset = train_dataset.repeat()
# train_dataset = train_dataset.shuffle(2048, seed=sn)

# #!!! this is a must!!!
# train_dataset = train_dataset.batch(128) ### !!! very important!!! to match with the batch_size in model.fit

# # train_dataset = train_dataset.cache()
# train_dataset = train_dataset.prefetch(128)

# ResNet Model

## Function to build a Model

In [3]:
def BasicBlock(x, planes, stride=1):
    global blockID, stage
    expansion = 1
    x_skip=x
# def __init__(self, in_planes, planes, stride=1):
#     super(BasicBlock, self).__init__()
    # self.conv1 = nn.Conv2d(
    #     in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    x=ZeroPadding2D(padding=(1, 1))(x)
    x=Conv2D(planes, kernel_size=3, strides=stride, use_bias=False, name=f'{stage}-{blockID}-Bconv1')(x)

    # self.bn1 = nn.BatchNorm2d(planes)
    x = BatchNormalization(name=f'{stage}-{blockID}-Bbn1')(x)
    x = Activation(activations.relu)(x)

    # self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
    #                        stride=1, padding=1, bias=False)
    x = Conv2D(planes, kernel_size=3, strides=1,  #as stride and kernel_size are fix
                padding="same", use_bias=False, name=f'{stage}-{blockID}-Bconv2')(x)   #padding='same' e.q. =1

    # self.bn2 = nn.BatchNorm2d(planes)
    x = BatchNormalization(name=f'{stage}-{blockID}-Bbn2')(x)

    # self.shortcut = nn.Sequential()

    # if stride != 1 or in_planes != self.expansion*planes:
    if stride != 1:    
        # self.shortcut = nn.Sequential(
        #     nn.Conv2d(in_planes, self.expansion*planes,
        #               kernel_size=1, stride=stride, bias=False),
        #     nn.BatchNorm2d(self.expansion*planes)
        # )
        x_skip = Conv2D(expansion*planes,kernel_size=1, strides=stride, 
                        use_bias=False, name=f'{stage}-{blockID}-Bconv3')(x_skip)
        x_skip = BatchNormalization(name=f'{stage}-{blockID}-Bbn3')(x_skip)            
        
    x = Add()([x,x_skip])
    # out = F.relu(out)
    out = Activation(activations.relu)(x)
    blockID=blockID+1
    return out

In [4]:
def make_layer(x, block, planes, num_blocks, stride, expansion):
    global stage, blockID
    strides = [stride] + [1]*(num_blocks-1)
    print("num of blocks [stride]:" + str(strides))
    for stride in strides:
        x = block(x, planes, stride)
        # in_planes = planes * expansion
    stage=stage+1
    blockID=1 # reset for next stage
    return x

def ResNet(block, num_blocks, num_classes=10, expansion=1):
    global blockID, stage
    blockID=1
    stage=1
    input_im = tf.keras.Input(shape=(train_images.shape[1], train_images.shape[2], train_images.shape[3]))

    x = Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=False) (input_im)

    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = make_layer(x, block, 64, num_blocks[0], 1, expansion)
    x = make_layer(x, block, 128, num_blocks[1], 2, expansion)
    x = make_layer(x, block, 256, num_blocks[2], 2, expansion)
    x = make_layer(x, block, 512, num_blocks[3], 2, expansion)
    
    x = AveragePooling2D(pool_size=4)(x)
    x = Flatten()(x)
    x = Dense(num_classes)(x)

    model = Model(inputs=input_im, outputs=x, name='Resnet')
    return model


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

# Callbacks

In [5]:
def resetLR(epoch, lr):
    if lr < 1e-5:
        lr = 1e-3

    print('learn rate:' + str(lr))
    return lr

lrdecay1 = tf.keras.callbacks.LearningRateScheduler(resetLR) # learning rate decay
lrdecay1

In [6]:
from math import pi,cos
PlateauStep_prev=0 
IniLr=0.01 
LrHist=np.array([])
def decayed_learning_rate(step, lr): 
    global PlateauStep_prev, IniLr, LrHist
    initial_learning_rate=IniLr
    decay_steps=200
    alpha=0
    currentLr=lr
    currentstep=step
    
    step = step - PlateauStep_prev
    step = min(step, decay_steps)
    cosine_decay = 0.5 * (1 + tf.cos(pi * step / decay_steps))
    decayed = (1 - alpha) * cosine_decay + alpha 
    lr=initial_learning_rate * decayed
    
    if currentLr<lr*(0.5+0.2): # 0.5 is the factor of tf.keras.callbacks.ReduceLROnPlateau
        PlateauStep_prev=currentstep
        IniLr=currentLr
        initial_learning_rate=IniLr
        step = step - PlateauStep_prev
        step = min(step, decay_steps)
        cosine_decay = 0.5 * (1 + tf.cos(pi * step / decay_steps))
        decayed = (1 - alpha) * cosine_decay + alpha 
        lr=initial_learning_rate * decayed
        
    print('learn rate:' + str(np.array(lr)))
    LrHist=np.block([LrHist,lr])
    return lr

CBcosLRdecay = tf.keras.callbacks.LearningRateScheduler(decayed_learning_rate)
CBcosLRdecay

# Compile Model

In [7]:
mod = ResNet18()
mod.summary()

num of blocks [stride]:[1, 1]
num of blocks [stride]:[2, 1]
num of blocks [stride]:[2, 1]
num of blocks [stride]:[2, 1]
Model: "Resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 64)   0           batch_n

In [8]:
#use categorical_crossentropy since the label is one-hot encoded
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
opt = SGD(momentum=0.9,decay = 1e-04) #

mod = ResNet18()
# # with tpu_strategy.scope():
# model.compile(optimizer = opt,
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), # False if softmax.
#               metrics=["accuracy"]) 

# use SparseCategoricalCrossentropy if labels is not encoded
mod.compile(optimizer = opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # use from_logits=True if no sotfmax
              metrics=["accuracy"])

num of blocks [stride]:[1, 1]
num of blocks [stride]:[2, 1]
num of blocks [stride]:[2, 1]
num of blocks [stride]:[2, 1]


# Run Training

In [10]:
es = EarlyStopping(patience= 40, restore_best_weights=True, monitor="val_accuracy", verbose=1)
lrdecay2=tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=7, verbose=1,
    mode='auto')
#I did not use cross validation, so the validate performance is not accurate.
# STEPS = len(X_train) / 128*2
history = mod.fit(
                    # x=train_images, y=train_labels,
                    # batch_size = 128, 
                    train_dataset,  #optimal 128 for GPU 
                    epochs=300, 
                    # validation_data=(test_images, test_labels),
                    validation_data=test_dataset,
                    callbacks=[CBcosLRdecay,es]
                    # callbacks=[lrdecay2,lrdecay1,es],
                   )

Epoch 1/300
learn rate:0.01
391/391 [==============================] - 48s 121ms/step - loss: 0.6483 - accuracy: 0.7971 - val_loss: 0.6629 - val_accuracy: 0.7788
Epoch 2/300
learn rate:0.009999383
391/391 [==============================] - 47s 120ms/step - loss: 0.4608 - accuracy: 0.8409 - val_loss: 0.7550 - val_accuracy: 0.7551
Epoch 3/300
learn rate:0.009997532
391/391 [==============================] - 47s 120ms/step - loss: 0.4125 - accuracy: 0.8557 - val_loss: 0.5070 - val_accuracy: 0.8282
Epoch 4/300
learn rate:0.009994449
391/391 [==============================] - 47s 120ms/step - loss: 0.3768 - accuracy: 0.8675 - val_loss: 0.6356 - val_accuracy: 0.7880
Epoch 5/300
learn rate:0.009990133
391/391 [==============================] - 47s 120ms/step - loss: 0.3471 - accuracy: 0.8787 - val_loss: 0.5437 - val_accuracy: 0.8203
Epoch 6/300
learn rate:0.009984586
391/391 [==============================] - 47s 120ms/step - loss: 0.3211 - accuracy: 0.8894 - val_loss: 0.5065 - val_accuracy: 

KeyboardInterrupt: ignored